In [ ]:
#imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
torch.__version__
import math
import time

In [ ]:
#global_variables
N,H,W,M,C,R,S,Sx,Sy=2,5,5,2,2,2,2,1,1
E,F=math.ceil((H-R)/Sx)+1,math.ceil((W-S)/Sy)+1
inp=torch.rand(N,2*C,H,W)
out=torch.zeros(N,M,E,F)
#wgt=torch.rand(M,C,R,S)
#bias=torch.zeros(N,M,E,F)

In [ ]:
index='6'

In [ ]:
#[N,H,W,C,R,S,C,M,Sx,Sy,g]
config={'0':[8,4,4,3,3,4,3,2,2,2,1],#small h,w,r,s,m, unequal strides, unequal r, s
        '1':[8,20,20,3,3,4,3,2,2,2,1],#big h,w;small r,s,m, unequal r, s, unequal strides
        '2':[8,20,20,3,10,10,3,2,2,2,1],#big h,w,r,s, equal, non-unity strides
        '3':[8,20,20,3,3,10,3,2,2,3,1],#big h,w;unequal r,s, unequal strides
        '4':[8,4,20,3,3,3,3,2,2,3,1],#unequal h,w;small r,s,m, unequal strides
        '5':[8,10,10,3,3,3,3,2,8,8,1],#equal non-unity wide strides 
        '6':[8,10,10,3,3,3,3,2,5,2,1],#unequal non-unity strides
        '7':[8,20,20,10,3,3,10,10,2,3,1],#big h,w,c, m;small r,s,m;unequal strides
        '8':[8,4,4,20,3,3,20,10,2,3,2],  #big c, m;small r,s,h,w unequal strides
        '9':[8,10,10,10,3,3,10,10,2,2,1],  #big h,w,m,c,non-unity strides
        }
N=config[index][0]
H=config[index][1]
W=config[index][2]
C=config[index][3]
R=config[index][4]
S=config[index][5]
C=config[index][6]
M=config[index][7]
Sx=config[index][8]
Sy=config[index][9]
g=config[index][10]
E,F=math.floor((H-R)/Sx)+1,math.floor((W-S)/Sy)+1
inp=torch.rand(N,C,H,W)
out=torch.zeros(N,M,E,F)
print('N:%d,H:%d,W:%d,C:%d;R:%d,S:%d,C:%d,M:%d;Sx:%d,Sy:%d,g:%d'%(N,H,W,C,R,S,C,M,Sx,Sy,g))

In [ ]:
#0.nn.conv2d
i_max=100
avg_time=0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
conv0=nn.Conv2d(C,M,kernel_size=(R,S),stride=(Sx,Sy),groups=g).to(device)
inp=inp.to(device)
for i in range(i_max):
    start_time=time.time()
    out0=conv0(inp)
    end_time=time.time()
    avg_time+=end_time-start_time
print('Total_time-1:',avg_time/i_max)
wgt0=conv0.weight
bias0=conv0.bias

In [ ]:
conv1=nn.Conv2d(2*C,12,kernel_size=(R,S),stride=(Sx,Sy),groups=2*C,bias=None)
out_=conv1(inp)
print(out_.shape)
print(inp.shape)
print(conv1.weight.shape)
#print(conv1.bias.shape)

In [ ]:
wit=conv1.weight
outo=torch.Tensor([])
for g in range(3):
    wit_=wit[g*4:g*4+4,:,:,:]#.reshape(1,-1,2,2)
    outo=torch.cat((outo,F.conv2d(inp,wit_,None,groups=4)),axis=1)

In [ ]:
print(outo,out_)

In [ ]:
outi=F.conv2d(inp,wit,bias=None)

In [ ]:
print(outi.shape)

In [ ]:
print('Equal1: ',torch.all(torch.abs(out_-outo)<9.6e-1))

In [ ]:
print(out_)

In [ ]:
print(outi)

In [ ]:
print(out0.shape,wgt0.shape,bias0.shape,inp.shape)
print(out0.dtype,out.dtype,bias0.dtype,wgt0.dtype,inp.dtype)

In [ ]:
#1.direct_convolution
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
avg_time=0
i_max=1
inp1=inp.clone().to(device)
wgt1=wgt0.clone().to(device)
bias1=bias0.clone().to(device)
avg_time=0
for i in range(i_max):
    out1=out.clone().to(device)
    start_time=time.time()
    for n in range(N):
        for e in range(E):
            for f in range(F):
                for m in range(M):
                    if e*Sx+R<H and f*Sy+S<W:
                        o=out1[n][m][e][f]
                        for c in range(C):
                            for r in range(R):
                                for s in range(S):
                                    o+=inp1[n][c][Sx*e+r][Sy*f+s]*wgt1[m][c][r][s]
                        out1[n][m][e][f]=o+bias1[m]
    end_time=time.time()
    avg_time+=end_time-start_time
print('Total_time0:',(avg_time)/i_max)
print('Equal0: ',torch.all(torch.eq(out0, out1)))
print('Equal1: ',torch.all(torch.abs(out0-out1)<2))

In [ ]:
print(out1.shape,out0.shape)

In [ ]:
#2.im2col_convolution
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
avg_time=0
i_max=2
inp2=inp.clone().to(device)
wgt2=wgt0.clone().to(device)
bias2=bias0.clone().to(device)
for i in range(i_max):
    r_inp=torch.Tensor([]).to(device)
    out2=torch.Tensor([]).to(device)
    start_time=time.time()
    r_wgt=wgt2.view(M,-1)
    for e in range(E):
        for f in range(F):
            r_inp=torch.cat((r_inp,inp2[:,:,Sx*e:Sx*e+R,Sy*f:Sy*f+S].reshape(N,-1).T),axis=-1)
    r_out=torch.matmul(r_wgt,r_inp)+torch.unsqueeze(bias0,axis=-1)  
    for n in range(N):
        out2=torch.cat((out2,r_out[:,n::N].reshape(1,-1,E,F)))
    end_time=time.time()
    avg_time+=end_time-start_time
print('Equal2: ',torch.all(torch.abs(out0-out2)<1e-5))
print('Total_time2:',avg_time/i_max)

In [ ]:
#winograd_conv
import sys
from torch import tensor

"""
author: Adam Dziedzic ady@uchicago.edu
Based on the paper: https://arxiv.org/abs/1509.09308
"""


class Winograd(object):
    B = tensor(
        [[1.0, 0.0, 0.0, 0.0],
         [0.0, 1.0, -1.0, 1.0],
         [-1.0, 1.0, 1.0, 0.0],
         [0.0, 0.0, 0.0, -1.0]]).to(device)
    B_T = B.transpose(1, 0)
    G = tensor(
        [[1.0, 0.0, 0.0],
         [0.5, 0.5, 0.5],
         [0.5, -0.5, 0.5],
         [0.0, 0.0, 1.0]]).to(device)
    G_T = G.transpose(1, 0)
    A = tensor([[1.0, 0.0],
                [1.0, 1.0],
                [1.0, -1.0],
                [0.0, -1.0]]).to(device)
    A_T = A.transpose(1, 0)

    def __init__(self, filter_value=None):
        super(Winograd, self).__init__()

        if filter_value is not None:
            self.filter = filter_value

    @staticmethod
    def forward(input, filter):
        """
        Compute Winograd convolution.
        :param input:
        :param filter:
        :return: output
        """
        N, C, H, W = input.size()
        K, Cprime, r, rprime = filter.size()
        assert H == W
        assert r == rprime
        assert C == Cprime
        m = 2
        a = m + r - 1
        # TODO pad with zeros the input for perfect tiling and slice the output.
        overlap = r - 1
        if (H >= 4 and H % 2 == 0) is False:
            raise Exception("Only input for perfect tiling is supported.")
        input = torch.transpose(input, 0, 1)
        assert input.size() == (C, N, H, W)
        # ntile = int(math.ceil(H//a))
        # P = N * ntile * ntile
        T = (W - a) // overlap + 1  # tiles_per_channel
        P = N * T * T
        U = torch.zeros(K, C, a, a).to(device)
        V = torch.zeros(C, P, a, a).to(device)
        for k in range(K):
            for c in range(C):
                U[k, c] = torch.matmul(Winograd.G,
                                       torch.matmul(filter[k, c], Winograd.G_T))
        for n in range(N):
            for tH in range(T):
                for tW in range(T):
                    for c in range(C):
                        b = n * (T * T) + tH * T + tW
                        vH = tH * (r - 1)
                        vW = tW * (r - 1)
                        V[c, b] = torch.matmul(Winograd.B_T, torch.matmul(
                            input[c, n, vH:vH + a, vW:vW + a], Winograd.B))
        M = torch.zeros(K, P, a, a).to(device)
        for k in range(K):
            for b in range(P):
                for c in range(C):
                    M[k, b] += U[k, c] * V[c, b]
        # M = torch.matmul(U, V)
        out_size = H - r + 1
        Y = torch.zeros(K, N, out_size, out_size).to(device)
        for k in range(K):
            for n in range(N):
                for tH in range(T):
                    for tW in range(T):
                        b = n * (T * T) + tH * T + tW
                        oH = tH * m
                        oW = tW * m
                        Y[k, n, oH:oH + m, oW:oW + m] = torch.matmul(
                            Winograd.A_T, torch.matmul(M[k, b], Winograd.A))

        Y = torch.transpose(Y, 0, 1)
        return Y

    @staticmethod
    def winograd_F_2_3(input, filter):
        """
        Compute winograd convolution with output of size 2x2 and filter of size
        3x3.
        :param input: 4x4
        :param filter: 3x3
        :return: 2x2
        """
        U = torch.matmul(Winograd.G, torch.matmul(filter, Winograd.G_T))
        V = torch.matmul(Winograd.B_T, torch.matmul(input, Winograd.B))
        return torch.matmul(Winograd.A_T, torch.matmul(U * V, Winograd.A))

    @staticmethod
    def winograd_F_1_3(input, filter):
        """
        Compute winograd convolution with output of size 1x1 and filter of size
        3x3. Input size is 3x3.
        :param input: 3x3
        :param filter: 3x3
        :return: 2x2
        """
        return input * filter


In [ ]:
#3.winograd_convolution
#Winograd.winograd_F_2_3(x, y)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
avg_time=0
i_max=2
inp3=inp.clone().to(device)
wgt3=wgt0.clone().to(device)
bias3=bias0.clone().to(device)
for i in range(i_max):
    start_time=time.time()
    out3=Winograd.forward(inp3,wgt3)#+bias3
    end_time=time.time()
    avg_time+=(end_time-start_time)
print('Total_time3:',avg_time/i_max)
print('Equal2: ',torch.all(torch.abs(out0-out3)<1))


In [ ]:
#winograd_conv2
from __future__ import print_function
from sympy import symbols, Matrix, Poly, zeros, eye, Indexed, simplify, IndexedBase, init_printing, pprint
from operator import mul
from functools import reduce

def At(a,m,n):
    return Matrix(m, n, lambda i,j: a[i]**j)

def A(a,m,n):
    return At(a, m-1, n).row_insert(m-1, Matrix(1, n, lambda i,j: 1 if j==n-1 else 0))

def T(a,n):
    return Matrix(Matrix.eye(n).col_insert(n, Matrix(n, 1, lambda i,j: -a[i]**n)))

def Lx(a,n):
    x=symbols('x')
    return Matrix(n, 1, lambda i,j: Poly((reduce(mul, ((x-a[k] if k!=i else 1) for k in range(0,n)), 1)).expand(basic=True), x))

def F(a,n):
    return Matrix(n, 1, lambda i,j: reduce(mul, ((a[i]-a[k] if k!=i else 1) for k in range(0,n)), 1))

def Fdiag(a,n):
    f=F(a,n)
    return Matrix(n, n, lambda i,j: (f[i,0] if i==j else 0))

def FdiagPlus1(a,n):
    f = Fdiag(a,n-1)
    f = f.col_insert(n-1, zeros(n-1,1))
    f = f.row_insert(n-1, Matrix(1,n, lambda i,j: (1 if j==n-1 else 0)))
    return f

def L(a,n):
    lx = Lx(a,n)
    f = F(a, n)
    return Matrix(n, n, lambda i,j: lx[i, 0].nth(j)/f[i]).T

def Bt(a,n):
    return L(a,n)*T(a,n)

def B(a,n):
    return Bt(a,n-1).row_insert(n-1, Matrix(1, n, lambda i,j: 1 if j==n-1 else 0))

FractionsInG=0
FractionsInA=1
FractionsInB=2
FractionsInF=3

def cookToomFilter(a,n,r,fractionsIn=FractionsInG):
    alpha = n+r-1
    f = FdiagPlus1(a,alpha)
    if f[0,0] < 0:
        f[0,:] *= -1
    if fractionsIn == FractionsInG:
        AT = A(a,alpha,n).T
        G = (A(a,alpha,r).T*f**(-1)).T
        BT = f * B(a,alpha).T
    elif fractionsIn == FractionsInA:
        BT = f * B(a,alpha).T
        G = A(a,alpha,r)
        AT = (A(a,alpha,n)).T*f**(-1)
    elif fractionsIn == FractionsInB:
        AT = A(a,alpha,n).T
        G = A(a,alpha,r)
        BT = B(a,alpha).T
    else:
        AT = A(a,alpha,n).T
        G = A(a,alpha,r)
        BT = f * B(a,alpha).T
    return (AT,G,BT,f)


def filterVerify(n, r, AT, G, BT):

    alpha = n+r-1

    di = IndexedBase('d')
    gi = IndexedBase('g')
    d = Matrix(alpha, 1, lambda i,j: di[i])
    g = Matrix(r, 1, lambda i,j: gi[i])

    V = BT*d
    U = G*g
    M = U.multiply_elementwise(V)
    Y = simplify(AT*M)

    return Y

def convolutionVerify(n, r, B, G, A):

    di = IndexedBase('d')
    gi = IndexedBase('g')

    d = Matrix(n, 1, lambda i,j: di[i])
    g = Matrix(r, 1, lambda i,j: gi[i])

    V = A*d
    U = G*g
    M = U.multiply_elementwise(V)
    Y = simplify(B*M)

    return Y

def showCookToomFilter(a,n,r,fractionsIn=FractionsInG):

    AT,G,BT,f = cookToomFilter(a,n,r,fractionsIn)

    print ("AT = ")
    pprint(AT)
    print ("")

    print ("G = ")
    pprint(G)
    print ("")

    print ("BT = ")
    pprint(BT)
    print ("")

    if fractionsIn != FractionsInF:
        print ("FIR filter: AT*((G*g)(BT*d)) =")
        pprint(filterVerify(n,r,AT,G,BT))
        print ("")

    if fractionsIn == FractionsInF:
        print ("fractions = ")
        pprint(f)
        print ("")

def showCookToomConvolution(a,n,r,fractionsIn=FractionsInG):

    AT,G,BT,f = cookToomFilter(a,n,r,fractionsIn)

    B = BT.transpose()
    A = AT.transpose()

    print ("A = ")
    pprint(A)
    print ("")

    print ("G = ")
    pprint(G)
    print ("")

    print ("B = ")
    pprint(B)
    print ("")

    if fractionsIn != FractionsInF:
        print ("Linear Convolution: B*((G*g)(A*d)) =")
        pprint(convolutionVerify(n,r,B,G,A))
        print ("")

    if fractionsIn == FractionsInF:
        print ("fractions = ")
        pprint(f)
        print ("")

In [ ]:
inp3=torch.randn(8,3,6,6)
wgt3=torch.randn(8,3,2,2)
conv3=nn.Conv2d(3,8,kernel_size=(2,2))
wgt3=conv3.weight
bias3=conv3.bias
out3=conv3(inp3)

In [ ]:
wgt3.shape

In [ ]:
gg=[[1/4,0],[-1/6,-1/6],[-1/6,1/6],[1/24,1/12],[1/24,-1/12],[0,1]]
gg=torch.Tensor(gg)
bb=[[4,0,0,0,0,0],[0,-4,4,-2,2,4],[-5,-4,-4,-1,-1,0],[0,1,-1,2,-2,-5],[1,1,1,1,1,0],[0,0,0,0,0,1]]
bb=torch.Tensor(bb)
#print(gg.shape,wgt3.shape,torch.matmul(gg,wgt3).shape)
U=torch.matmul(torch.matmul(gg,wgt3),gg.T)
V=torch.matmul(torch.matmul(bb.T,inp3),bb)
aa=[[1,0,0,0,0],[1,1,1,1,1],[1,-1,1,-1,1],[1,2,4,8,16],[1,-2,4,-8,16],[0,0,0,0,1]]
aa=torch.Tensor(aa)
out_=torch.matmul(torch.matmul(aa.T,U.mul(V)),aa)+bias3.reshape(8,1,1,1)

In [ ]:
out_

In [ ]:
out

In [ ]:
showCookToomConvolution((0,1,-1,2,-2),5,2)

In [ ]:
#fft_conv
from functools import partial
from typing import Tuple, Union, Iterable

import torch
from torch import nn, Tensor
from torch.fft import rfftn, irfftn
import torch.nn.functional as f


def complex_matmul(a: Tensor, b: Tensor, groups: int = 1) -> Tensor:
    """Multiplies two complex-valued tensors."""
    # Scalar matrix multiplication of two tensors, over only the first channel
    # dimensions. Dimensions 3 and higher will have the same shape after multiplication.
    # We also allow for "grouped" multiplications, where multiple sections of channels
    # are multiplied independently of one another (required for group convolutions).
    scalar_matmul = partial(torch.einsum, "agc..., gbc... -> agb...")
    a = a.view(a.size(0), groups, -1, *a.shape[2:])
    b = b.view(groups, -1, *b.shape[1:])

    # Compute the real and imaginary parts independently, then manually insert them
    # into the output Tensor.  This is fairly hacky but necessary for PyTorch 1.7.0,
    # because Autograd is not enabled for complex matrix operations yet.  Not exactly
    # idiomatic PyTorch code, but it should work for all future versions (>= 1.7.0).
    real = scalar_matmul(a.real, b.real) - scalar_matmul(a.imag, b.imag)
    imag = scalar_matmul(a.imag, b.real) + scalar_matmul(a.real, b.imag)
    c = torch.zeros(real.shape, dtype=torch.complex64, device=a.device)
    c.real, c.imag = real, imag

    return c.view(c.size(0), -1, *c.shape[3:])


def to_ntuple(val: Union[int, Iterable[int]], n: int) -> Tuple[int, ...]:
    """Casts to a tuple with length 'n'.  Useful for automatically computing the
    padding and stride for convolutions, where users may only provide an integer.
    Args:
        val: (Union[int, Iterable[int]]) Value to cast into a tuple.
        n: (int) Desired length of the tuple
    Returns:
        (Tuple[int, ...]) Tuple of length 'n'
    """
    if isinstance(val, Iterable):
        out = tuple(val)
        if len(out) == n:
            return out
        else:
            raise ValueError(f"Cannot cast tuple of length {len(out)} to length {n}.")
    else:
        return n * (val,)


def fft_conv(
    signal: Tensor,
    kernel: Tensor,
    bias: Tensor = None,
    padding: Union[int, Iterable[int]] = 0,
    stride: Union[int, Iterable[int]] = 1,
    groups: int = 1,
) -> Tensor:
    """Performs N-d convolution of Tensors using a fast fourier transform, which
    is very fast for large kernel sizes. Also, optionally adds a bias Tensor after
    the convolution (in order ot mimic the PyTorch direct convolution).
    Args:
        signal: (Tensor) Input tensor to be convolved with the kernel.
        kernel: (Tensor) Convolution kernel.
        bias: (Tensor) Bias tensor to add to the output.
        padding: (Union[int, Iterable[int]) Number of zero samples to pad the
            input on the last dimension.
        stride: (Union[int, Iterable[int]) Stride size for computing output values.
    Returns:
        (Tensor) Convolved tensor
    """
    # Cast padding & stride to tuples.
    padding_ = to_ntuple(padding, n=signal.ndim - 2)
    stride_ = to_ntuple(stride, n=signal.ndim - 2)

    # Pad the input signal & kernel tensors
    signal_padding = [p for p in padding_[::-1] for _ in range(2)]
    signal = f.pad(signal, signal_padding)

    # Because PyTorch computes a *one-sided* FFT, we need the final dimension to
    # have *even* length.  Just pad with one more zero if the final dimension is odd.
    if signal.size(-1) % 2 != 0:
        signal_ = f.pad(signal, [0, 1])
    else:
        signal_ = signal

    kernel_padding = [
        pad
        for i in reversed(range(2, signal_.ndim))
        for pad in [0, signal_.size(i) - kernel.size(i)]
    ]
    padded_kernel = f.pad(kernel, kernel_padding)

    # Perform fourier convolution -- FFT, matrix multiply, then IFFT
    # signal_ = signal_.reshape(signal_.size(0), groups, -1, *signal_.shape[2:])
    signal_fr = rfftn(signal_, dim=tuple(range(2, signal.ndim)))
    kernel_fr = rfftn(padded_kernel, dim=tuple(range(2, signal.ndim)))

    kernel_fr.imag *= -1
    output_fr = complex_matmul(signal_fr, kernel_fr, groups=groups)
    output = irfftn(output_fr, dim=tuple(range(2, signal.ndim)))

    # Remove extra padded values
    crop_slices = [slice(0, output.size(0)), slice(0, output.size(1))] + [
        slice(0, (signal.size(i) - kernel.size(i) + 1), stride_[i - 2])
        for i in range(2, signal.ndim)
    ]
    output = output[crop_slices].contiguous()

    # Optionally, add a bias term before returning.
    if bias is not None:
        bias_shape = tuple([1, -1] + (signal.ndim - 2) * [1])
        output += bias.view(bias_shape)

    return output


class _FFTConv(nn.Module):
    """Base class for PyTorch FFT convolution layers."""

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size: Union[int, Iterable[int]],
        padding: Union[int, Iterable[int]] = 0,
        stride: Union[int, Iterable[int]] = 1,
        groups: int = 1,
        bias: bool = True,
        ndim: int = 1,
    ):
        """
        Args:
            in_channels: (int) Number of channels in input tensors
            out_channels: (int) Number of channels in output tensors
            kernel_size: (Union[int, Iterable[int]) Square radius of the kernel
            padding: (Union[int, Iterable[int]) Number of zero samples to pad the
                input on the last dimension.
            stride: (Union[int, Iterable[int]) Stride size for computing output values.
            bias: (bool) If True, includes bias, which is added after convolution
        """
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.stride = stride
        self.groups = groups
        self.use_bias = bias

        if in_channels % 2 != 0:
            raise ValueError(
                "'in_channels' must be divisible by 'groups'."
                f"Found: in_channels={in_channels}, groups={groups}."
            )
        if out_channels % 2 != 0:
            raise ValueError(
                "'out_channels' must be divisible by 'groups'."
                f"Found: out_channels={out_channels}, groups={groups}."
            )

        kernel_size = to_ntuple(kernel_size, ndim)
        self.weight = nn.Parameter(
            torch.randn(out_channels, in_channels // groups, *kernel_size)
        )
        self.bias = nn.Parameter(torch.randn(out_channels,)) if bias else None

    def forward(self, signal):
        return fft_conv(
            signal,
            self.weight,
            bias=self.bias,
            padding=self.padding,
            stride=self.stride,
            groups=self.groups,
        )


FFTConv1d = partial(_FFTConv, ndim=1)
FFTConv2d = partial(_FFTConv, ndim=2)
FFTConv3d = partial(_FFTConv, ndim=3)

In [ ]:
#4.fft_convolution
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
avg_time=0
i_max=5
inp4=inp.clone().to(device)
wgt4=wgt0.clone().to(device)
bias4=bias0.clone().to(device)
for i in range(i_max):
    start_time=time.time()
    out4 = fft_conv(inp4, wgt4, bias=bias4,stride=(Sx,Sy),groups=g)
    end_time=time.time()
    avg_time+=end_time-start_time
print('Total_time4:',avg_time/i_max)
print('Equal4: ',torch.all(torch.abs(out0-out4)<1e-5))


In [ ]:
print(out4.shape,out0.shape)

In [ ]:
print('Equal4: ',torch.all(torch.abs(out3-out_-bias_)<1e-5))

In [ ]:
print(out_.shape)